In [1]:
import os, urllib.request
import numpy as np, pandas as pd
from typing import Dict, Tuple
from scipy import sparse

from implicit.nearest_neighbours import bm25_weight
from implicit.nearest_neighbours import CosineRecommender
from implicit.als import AlternatingLeastSquares

/Users/a202304035/RecSys_project/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --------------------------------------------------
# A. MovieLens 100k 다운로드/로드
#   - 파일이 없으면 자동 다운로드 (~/ml-100k/)
#   - u.data: user_id, item_id, rating, timestamp (tab-separated)
# --------------------------------------------------
def load_movielens_100k(data_dir: str = os.path.expanduser("~/ml-100k")) -> pd.DataFrame:
    os.makedirs(data_dir, exist_ok=True)
    ratings_path = os.path.join(data_dir, "u.data")

    if not os.path.exists(ratings_path):
        print("📥 Downloading MovieLens 100k ...")
        url = "https://files.grouplens.org/datasets/movielens/ml-100k/u.data"
        # 직접 파일만 받는 간단 버전 (압축 해제 과정 없이)
        urllib.request.urlretrieve(url, ratings_path)

    df = pd.read_csv(
        ratings_path, 
        sep="\t", 
        names=["user_id", "item_id", "rating", "timestamp"], 
        engine="python"
    )
    # 문자열로 통일(매핑 안정)
    df["user_id"] = df["user_id"].astype(str)
    df["item_id"] = df["item_id"].astype(str)
    return df


In [3]:
#   - 각 유저의 최신 1개를 test로, 나머지는 train으로 사용
# --------------------------------------------------
def train_test_split_loov(ratings: pd.DataFrame) -> tuple[pd.DataFrame, pd.DataFrame]:
    ratings = ratings.copy()
    ratings["ts_rank"] = ratings.groupby("user_id")["timestamp"].rank(method="first", ascending=False)
    test = ratings[ratings["ts_rank"] == 1].drop(columns=["ts_rank"])
    train = ratings[ratings["ts_rank"] > 1].drop(columns=["ts_rank"])
    return train, test

In [4]:
# C. ID 매핑 + CSR 생성 (user x item)
#   - implicit은 학습 시 item-user를 기대하므로 전치해서 전달
#   - MovieLens는 explicit(평점)이므로, rating을 confidence로 사용
# --------------------------------------------------
def build_id_maps(train_df: pd.DataFrame):
    users = train_df["user_id"].unique()
    items = train_df["item_id"].unique()
    u2i = {u:i for i,u in enumerate(users)}
    it2i = {it:i for i,it in enumerate(items)}
    return u2i, it2i, np.array(users), np.array(items)

def build_ui_csr(df: pd.DataFrame, user2idx: dict, item2idx: dict, use_rating=True) -> sparse.csr_matrix:
    d = df[["user_id","item_id","rating"] if "rating" in df.columns else ["user_id","item_id"]].copy()
    d["u"] = d["user_id"].map(user2idx)
    d["i"] = d["item_id"].map(item2idx)
    d = d.dropna(subset=["u","i"])
    rows = d["u"].astype(int).to_numpy()
    cols = d["i"].astype(int).to_numpy()
    vals = (d["rating"].astype(np.float32).to_numpy() if use_rating and "rating" in d.columns
            else np.ones(len(d), dtype=np.float32))
    UI = sparse.csr_matrix((vals, (rows, cols)), shape=(len(user2idx), len(item2idx)), dtype=np.float32)
    return UI

In [5]:
# D. 모델 학습 (Cosine KNN / ALS)
# --------------------------------------------------
def fit_cosine_knn(item_user: sparse.csr_matrix, K:int=200) -> CosineRecommender:
    model = CosineRecommender(K=K)
    model.fit(item_user)   # expect item-user
    return model

def fit_als(user_item: sparse.csr_matrix, factors=64, reg=1e-2, iters=20) -> AlternatingLeastSquares:
    als = AlternatingLeastSquares(factors=factors, regularization=reg, iterations=iters)
    als.fit(user_item)     # expect item-user
    return als


In [6]:
# E. 추천/유사아이템/간단 평가
# --------------------------------------------------
from scipy import sparse

def recommend_topn(model, UI_bm25, user_id, N=10):
    user_row = UI_bm25[user_id]  # 1 x n_items
    ids, scores = model.recommend(
        user_id,
        user_items=user_row,
        N=N,
        filter_already_liked_items=True
    )
    return list(zip(ids, scores))

def knn_recommend_topn(model, UI_weighted: sparse.csr_matrix, user_internal_id: int, N: int = 10):
    user_row = UI_weighted[user_internal_id]
    ids, scores = model.recommend(
        user_internal_id,
        user_items=user_row,                  
        N=N,
        filter_already_liked_items=True
    )
    return list(zip(ids, scores))

def similar_items_topn(model, item_internal_id: int, N:int=10):
    ids, scores = model.similar_items(item_internal_id, N=N)
    return list(zip(ids, scores))

In [7]:
# 실행 part
ratings = load_movielens_100k()
train_df, test_df = train_test_split_loov(ratings)
print(f"train: {train_df.shape}, test: {test_df.shape}")

train: (99057, 4), test: (943, 4)


In [8]:
train_df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
print(train_df['user_id'].nunique())
print(train_df['item_id'].nunique())

943
1680


In [10]:
# 매핑/행렬
user2idx, item2idx, idx2user, idx2item = build_id_maps(train_df)
UI = build_ui_csr(train_df, user2idx, item2idx, use_rating=True)

# 가중(둘 중 하나 선택; 보통 BM25 권장)
UI_bm25  = bm25_weight(UI).tocsr()
# UI_tfidf = tfidf_weight(UI).tocsr()

# implicit은 item-user 기대 → 전치
IU_bm25 = UI_bm25.T.tocsr()

# 모델 학습
knn = fit_cosine_knn(IU_bm25, K=200)
als = fit_als(UI_bm25, factors=64, reg=1e-2, iters=20)

# 데모: 유저/아이템 임의 선택
u0 = 0
i0 = 0
print("KNN recommend(u0):", recommend_topn(knn, UI_bm25, u0, N=10)[:5])
print("KNN similar_items(i0):", similar_items_topn(knn, i0, N=10)[:5])
print("ALS recommend(u0):", recommend_topn(als, UI_bm25, u0, N=10)[:5])

/Users/a202304035/RecSys_project/.venv/lib/python3.11/site-packages/implicit/utils.py:164: ParameterWarning: Method expects CSR input, and was passed coo_matrix instead. Converting to CSR took 0.0008740425109863281 seconds
  warnings.warn(
100%|██████████| 20/20 [00:00<00:00, 31.27it/s]

KNN recommend(u0): [(np.int32(584), np.float64(64.21088724546001)), (np.int32(50), np.float64(63.18584760933893)), (np.int32(420), np.float64(62.72465281039928)), (np.int32(534), np.float64(60.02447432974939)), (np.int32(65), np.float64(59.07429995913074))]
KNN similar_items(i0): [(np.int32(0), np.float64(1.0)), (np.int32(320), np.float64(0.28796264316884335)), (np.int32(569), np.float64(0.254302711272459)), (np.int32(264), np.float64(0.2489866512472389)), (np.int32(27), np.float64(0.2372621644461647))]
ALS recommend(u0): [(np.int32(70), np.float32(0.8081374)), (np.int32(867), np.float32(0.8030158)), (np.int32(33), np.float32(0.79736036)), (np.int32(360), np.float32(0.7612708)), (np.int32(242), np.float32(0.7294104))]


In [11]:
from typing import Dict, Tuple
import numpy as np
import pandas as pd
from scipy import sparse

def eval_loov_recall_at_k(
    model,
    UI_bm25: sparse.csr_matrix,
    test_df: pd.DataFrame,
    user2idx: Dict[str, int],
    item2idx: Dict[str, int],
    K: int = 10,
    filter_seen: bool = True,
) -> Tuple[float, int, int]:
    # 0) 공간 정합성(아이템 축)
    if hasattr(model, "item_factors"):
        n_items_model = model.item_factors.shape[0]
        if UI_bm25.shape[1] != n_items_model:
            raise ValueError(f"[SpaceMismatch] model items={n_items_model} vs UI cols={UI_bm25.shape[1]}")

    hits, total = 0, 0

    # 1) 유저 루프: 각 유저에 대해 반드시 '그 유저의 1행'만 전달
    for u, g in test_df.groupby("user_id"):
        if u not in user2idx:
            continue  # cold user
        uidx = user2idx[u]

        # 평가 대상 아이템(학습 매핑에 있는 것만)
        target_items = [item2idx[it] for it in g["item_id"].values if it in item2idx]
        if not target_items:
            continue  # cold item만 있으면 스킵

        # 2) user_row 한 행 추출 + out-of-range 방어
        user_row = UI_bm25[uidx]
        if hasattr(model, "item_factors"):
            n_items_model = model.item_factors.shape[0]
            if user_row.indices.size and user_row.indices.max() >= n_items_model:
                # 비정상 인덱스가 있으면 잘라내기(정상이라면 0건)
                mask = user_row.indices < n_items_model
                user_row = sparse.csr_matrix(
                    (user_row.data[mask], user_row.indices[mask], user_row.indptr.copy()),
                    shape=(1, n_items_model)
                )

        # 3) recommend 호출(문제 발생시 어느 유저에서 터졌는지 알기 위해 try/except)
        try:
            rec_ids, _ = model.recommend(
                uidx,
                user_items=user_row,
                N=K,
                filter_already_liked_items=filter_seen
            )
        except Exception as e:
            raise RuntimeError(f"[CrashAtUser] user='{u}' (uidx={uidx}) 동안 recommend() 실패: {e}")

        # 4) Hit 판정
        if set(rec_ids).intersection(target_items):
            hits += 1
        total += 1

    hit_rate = hits / total if total > 0 else 0.0
    return hit_rate, hits, total
